In [0]:
%python
from pyspark.sql import SparkSession
from pyspark.sql.functions import first, rand, count,col

spark = SparkSession.builder.appName("JoinsExample").getOrCreate()

dates = spark.sql("SELECT explode(sequence(DATE'2024-01-01', DATE'2024-03-24', INTERVAL 1 DAY)) as calendar_date")

c_id = spark.sql("SELECT explode(sequence(1,12000, 1)) as client_id")

types = spark.sql("SELECT concat('col_', colName) as col_name FROM (SELECT explode(sequence(1,20, 1)) as colName)")

dates = dates.repartition(99)
c_id = c_id.repartition(11)
types = types.repartition(1)

df_cartesian = (c_id.crossJoin(dates.select("calendar_date")).crossJoin(types.select("col_name")).select("client_id", "calendar_date", "col_name"))

df_cartesian = df_cartesian.withColumn("val", (rand() * 10).cast("int"))
df_grp1 = df_cartesian.groupBy("client_id", "calendar_date").pivot("col_name").agg(first("val").alias("val"))
df_grp2 = df_grp1.withColumnRenamed("client_id", "client_id_2")
df_grp2 = df_grp2.select(
    *[col(c).alias(f"{c}_2") for c in df_grp2.columns]
)


df_grp1.count()







Out[53]: 1008000

In [0]:
%python
df_inner = df_grp1.join(df_grp2, (df_grp1.client_id == df_grp2.client_id_2_2) & 
                                    (df_grp1.calendar_date == df_grp2.calendar_date_2), "inner")

df_left = df_grp1.join(df_grp2, (df_grp1.client_id == df_grp2.client_id_2_2) & 
                                   (df_grp1.calendar_date == df_grp2.calendar_date_2), "left")

display(df_inner.limit(10))
display(df_left.limit(10))
df_inner.count()


client_id,calendar_date,col_1,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_2,col_20,col_3,col_4,col_5,col_6,col_7,col_8,col_9,client_id_2_2,calendar_date_2,col_1_2,col_10_2,col_11_2,col_12_2,col_13_2,col_14_2,col_15_2,col_16_2,col_17_2,col_18_2,col_19_2,col_2_2,col_20_2,col_3_2,col_4_2,col_5_2,col_6_2,col_7_2,col_8_2,col_9_2
1,2024-01-01,0,1,3,5,0,2,2,8,9,9,4,4,1,4,6,4,1,9,7,4,1,2024-01-01,0,1,3,5,0,2,2,8,9,9,4,4,1,4,6,4,1,9,7,4
1,2024-01-02,1,2,9,2,9,0,8,7,1,5,7,0,3,7,1,8,4,4,7,3,1,2024-01-02,1,2,9,2,9,0,8,7,1,5,7,0,3,7,1,8,4,4,7,3
1,2024-01-03,1,5,6,2,9,5,0,8,8,6,8,4,2,5,3,8,7,3,2,2,1,2024-01-03,1,5,6,2,9,5,0,8,8,6,8,4,2,5,3,8,7,3,2,2
1,2024-01-04,1,7,4,1,5,7,2,7,3,4,1,9,5,2,5,0,1,8,6,2,1,2024-01-04,1,7,4,1,5,7,2,7,3,4,1,9,5,2,5,0,1,8,6,2
1,2024-01-05,1,0,6,8,0,9,3,6,8,2,7,0,8,8,3,1,8,3,9,5,1,2024-01-05,1,0,6,8,0,9,3,6,8,2,7,0,8,8,3,1,8,3,9,5
1,2024-01-06,1,0,7,1,4,9,1,6,7,5,0,1,0,6,7,1,8,0,2,3,1,2024-01-06,1,0,7,1,4,9,1,6,7,5,0,1,0,6,7,1,8,0,2,3
1,2024-01-07,4,6,4,2,9,6,9,0,2,3,5,3,6,2,5,5,4,4,5,7,1,2024-01-07,4,6,4,2,9,6,9,0,2,3,5,3,6,2,5,5,4,4,5,7
1,2024-01-08,0,2,9,7,3,7,5,8,1,6,2,3,5,3,1,0,9,2,2,7,1,2024-01-08,0,2,9,7,3,7,5,8,1,6,2,3,5,3,1,0,9,2,2,7
1,2024-01-09,2,0,2,7,1,1,9,2,2,7,3,3,4,3,2,8,6,0,3,2,1,2024-01-09,2,0,2,7,1,1,9,2,2,7,3,3,4,3,2,8,6,0,3,2
1,2024-01-10,2,5,2,5,8,1,0,0,2,7,8,1,8,3,5,8,0,2,8,7,1,2024-01-10,2,5,2,5,8,1,0,0,2,7,8,1,8,3,5,8,0,2,8,7


client_id,calendar_date,col_1,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_2,col_20,col_3,col_4,col_5,col_6,col_7,col_8,col_9,client_id_2_2,calendar_date_2,col_1_2,col_10_2,col_11_2,col_12_2,col_13_2,col_14_2,col_15_2,col_16_2,col_17_2,col_18_2,col_19_2,col_2_2,col_20_2,col_3_2,col_4_2,col_5_2,col_6_2,col_7_2,col_8_2,col_9_2
5324,2024-01-17,3,0,8,1,2,7,9,5,5,3,2,4,3,7,1,8,7,8,1,6,5324,2024-01-17,3,0,8,1,2,7,9,5,5,3,2,4,3,7,1,8,7,8,1,6
5324,2024-01-29,6,1,3,6,7,7,8,7,8,8,3,8,1,6,2,0,3,5,2,3,5324,2024-01-29,6,1,3,6,7,7,8,7,8,8,3,8,1,6,2,0,3,5,2,3
5324,2024-01-31,9,9,1,3,4,3,1,2,8,2,5,6,6,3,4,7,0,1,6,5,5324,2024-01-31,9,9,1,3,4,3,1,2,8,2,5,6,6,3,4,7,0,1,6,5
5324,2024-02-24,1,3,2,0,7,7,9,4,2,5,7,2,5,4,4,2,8,0,5,6,5324,2024-02-24,1,3,2,0,7,7,9,4,2,5,7,2,5,4,4,2,8,0,5,6
5324,2024-02-26,5,9,7,8,1,8,7,3,0,3,7,2,3,9,5,5,7,5,7,1,5324,2024-02-26,5,9,7,8,1,8,7,3,0,3,7,2,3,9,5,5,7,5,7,1
5324,2024-02-29,5,3,4,4,0,0,2,8,9,8,0,3,4,8,4,6,4,2,7,7,5324,2024-02-29,5,3,4,4,0,0,2,8,9,8,0,3,4,8,4,6,4,2,7,7
5324,2024-03-07,5,0,1,7,1,4,7,6,9,9,1,9,2,5,1,9,8,6,3,0,5324,2024-03-07,5,0,1,7,1,4,7,6,9,9,1,9,2,5,1,9,8,6,3,0
5324,2024-03-19,2,9,8,3,4,4,8,2,3,9,7,3,9,7,8,0,3,7,9,0,5324,2024-03-19,2,9,8,3,4,4,8,2,3,9,7,3,9,7,8,0,3,7,9,0
5324,2024-03-21,2,3,8,1,9,5,7,0,9,1,7,7,3,7,8,9,3,8,5,0,5324,2024-03-21,2,3,8,1,9,5,7,0,9,1,7,7,3,7,8,9,3,8,5,0
5324,2024-03-24,8,6,6,2,5,4,4,1,0,6,6,4,5,7,2,8,6,0,0,6,5324,2024-03-24,8,6,6,2,5,4,4,1,0,6,6,4,5,7,2,8,6,0,0,6


Out[55]: 1008000

zmienilem nazwy drugiej tabeli dodając '_2' - teraz mozemy je wg chęci podropować